# Proyecto | Base de Datos de Big Data

## Informacion del Equipo o Persona

### Equipo #20
1.- Julio Baltazar Colín - A01794476

2.- Alejandro González Almazán - A00517113

3.- Erik Morales Hinojosa - A01795110

4.- Kevin Alexis Valdez Pérez - A01750478

--------------------------------------------------------------------------------

## Instalacion de Librerias

In [ ]:
"""
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
"""
!pip install -q findspark

# Impotacion de Librerias

In [ ]:
# PySpark
import findspark
findspark.init()
findspark.find()
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, mean, stddev, min, max
from pyspark.sql.functions import col
"""
# omitir para ejecutar de forma local
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"
"""
#Librerias de codigo
import kagglehub
from pyspark.sql import functions as F
from pyspark.sql.functions import col

### Inicializar entorno PySpark

In [ ]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Analisis_Steam") \
    .getOrCreate()

spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

spark

### Lectura de datos

In [ ]:
file_path = kagglehub.dataset_download("najzeko/steam-reviews-2021")

100%|██████████| 2.97G/2.97G [00:31<00:00, 102MB/s] 

Extracting files...


In [ ]:
file_path= "D:\\source\\bagdata\\archive\\steam_reviews.csv"

In [ ]:
df = spark.read.option("header", "true") \
    .option("inferSchema", "true") \
    .option("multiLine", "true") \
    .option("sep", ",").option("escape", "\"").csv(file_path)

## Exploración inicial del dataset

In [ ]:
print("Primeras filas del dataset:")
df.show(5, truncate=100)

Primeras filas del dataset:
+---+------+------------------------+---------+--------+-----------------------------------------------------------+-----------------+-----------------+-----------+-------------+-----------+-------------------+-------------+--------------+-----------------+---------------------------+-----------------+----------------------+------------------+-----------------------+------------------------------+-------------------------+------------------+
|_c0|app_id|                app_name|review_id|language|                                                     review|timestamp_created|timestamp_updated|recommended|votes_helpful|votes_funny|weighted_vote_score|comment_count|steam_purchase|received_for_free|written_during_early_access|   author.steamid|author.num_games_owned|author.num_reviews|author.playtime_forever|author.playtime_last_two_weeks|author.playtime_at_review|author.last_played|
+---+------+------------------------+---------+--------+------------------------

In [ ]:
# Reemplazar "." por "_" en todas las columnas
new_columns = [col_name.replace(".", "_") for col_name in df.columns]
df = df.toDF(*new_columns)


In [ ]:
# Mostrar el esquema del DataFrame
print("\nEstructura del dataset:")
df.printSchema()


Estructura del dataset:
root
 |-- _c0: integer (nullable = true)
 |-- app_id: integer (nullable = true)
 |-- app_name: string (nullable = true)
 |-- review_id: integer (nullable = true)
 |-- language: string (nullable = true)
 |-- review: string (nullable = true)
 |-- timestamp_created: integer (nullable = true)
 |-- timestamp_updated: long (nullable = true)
 |-- recommended: boolean (nullable = true)
 |-- votes_helpful: long (nullable = true)
 |-- votes_funny: long (nullable = true)
 |-- weighted_vote_score: double (nullable = true)
 |-- comment_count: integer (nullable = true)
 |-- steam_purchase: boolean (nullable = true)
 |-- received_for_free: boolean (nullable = true)
 |-- written_during_early_access: boolean (nullable = true)
 |-- author_steamid: long (nullable = true)
 |-- author_num_games_owned: long (nullable = true)
 |-- author_num_reviews: long (nullable = true)
 |-- author_playtime_forever: double (nullable = true)
 |-- author_playtime_last_two_weeks: double (nullable = t

In [ ]:
# Estadísticas descriptivas generales
print("\nEstadísticas descriptivas:")
df.describe()


Estadísticas descriptivas:


summary,_c0,app_id,app_name,review_id,language,review,timestamp_created,timestamp_updated,votes_helpful,votes_funny,weighted_vote_score,comment_count,author_steamid,author_num_games_owned,author_num_reviews,author_playtime_forever,author_playtime_last_two_weeks,author_playtime_at_review,author_last_played
count,21747371,21747371,21747371,21747371,21747371,21713629,21747371,21747371,21747371,21747371,21747371,21747371,21747371,21747371,21747371,21747369,21747369,21721689,21747369
mean,1.0873687437602503E7,392818.0819021297,NULL,5.187499536142856E7,NULL,NaN,1.544432473641996E9,1.5475556641072469E9,404468.8552777713,126791.72557597881,0.16544240107804964,0.13087678506059422,7.656119829647547...,1011300.0967205645,404477.4800190791,16091.04968789558,155.5421350968938,8807.421419761604,1.5805050620579658E9
stddev,6277926.749125054,248097.74044015192,NULL,2.084267274558985E7,NULL,NaN,5.8121938249827236E7,5.689899216654692E7,1.333740761025968E9,2.333552590589009E7,0.24340060997837204,2.1993978415663555,3.177567912050335E8,2.1088291610977833E9,1.3337407610244334E9,37430.56775904549,730.0488048533861,23885.530542759574,4.6761775230105035E7
min,0,70,20XX,43,brazilian,.?m?xÚ•”ÇoA...,1290197836,1290197836,0,0,0.0,0,76561197960265730,0,1,0.0,0.0,1.0,0.0
max,21747375,1291340,三国群英传8 Heroes of ...,85218673,vietnamese,󰀐󰀐󰀐󰀐󰀐󰀐󰀐󰀐?...,1611426287,2283382797,4398046511106,4294967295,0.995986759662628,4893,76561199133265373,4398046511619,4398046511108,3744943.0,27039.0,3228103.0,1.611434142E9


##Caracterización de la población




In [ ]:
# Frecuencia de categorías
print("Recomendados")
df.groupBy("recommended").count().show()
print("Recibidos gratis")
df.groupBy("received_for_free").count().show()
print("Conteo por idioma")
df.groupBy("language").count().orderBy("count", ascending=False).show(25)

+-----------+--------+
|recommended|   count|
+-----------+--------+
|       true|19022610|
|      false| 2724761|
+-----------+--------+

+-----------------+--------+
|received_for_free|   count|
+-----------------+--------+
|             true|  688061|
|            false|21059310|
+-----------------+--------+

+----------+-------+
|  language|  count|
+----------+-------+
|   english|9635437|
|  schinese|3764967|
|   russian|2348900|
| brazilian| 837524|
|   spanish| 813320|
|    german| 752596|
|   turkish| 635868|
|   koreana| 613632|
|    french| 541751|
|    polish| 495529|
|  tchinese| 218203|
|     czech| 133980|
|   italian| 133307|
|      thai| 127503|
|  japanese|  81754|
|portuguese|  81386|
|   swedish|  80226|
|     dutch|  77555|
| hungarian|  71001|
|     latam|  70103|
|    danish|  55915|
|   finnish|  54712|
| norwegian|  36797|
|  romanian|  32730|
| ukrainian|  21169|
+----------+-------+
only showing top 25 rows



In [ ]:


numeric_columns = [
    "author_num_games_owned",
    "author_num_reviews",
    "author_playtime_forever",
    "votes_helpful"
]

# Para cada variable numérica, imprime una tabla resumen con estadísticas
for col_name in numeric_columns:
    df.select(col_name).summary("count", "mean", "stddev", "min", "max").show()

+-------+----------------------+
|summary|author_num_games_owned|
+-------+----------------------+
|  count|              21747371|
|   mean|    1011300.0967205645|
| stddev|  2.1088291610977833E9|
|    min|                     0|
|    max|         4398046511619|
+-------+----------------------+

+-------+--------------------+
|summary|  author_num_reviews|
+-------+--------------------+
|  count|            21747371|
|   mean|   404477.4800190791|
| stddev|1.3337407610244334E9|
|    min|                   1|
|    max|       4398046511108|
+-------+--------------------+

+-------+-----------------------+
|summary|author_playtime_forever|
+-------+-----------------------+
|  count|               21747369|
|   mean|      16091.04968789558|
| stddev|      37430.56775904549|
|    min|                    0.0|
|    max|              3744943.0|
+-------+-----------------------+

+-------+-------------------+
|summary|      votes_helpful|
+-------+-------------------+
|  count|           21747

In [ ]:

for col_name in numeric_columns:
    print(f"Resumen para: {col_name}")
    df.select(
        count(col(col_name)).alias(f"{col_name}_count"),
        mean(col(col_name)).alias(f"{col_name}_mean"),
        stddev(col(col_name)).alias(f"{col_name}_stddev"),
        min(col(col_name)).alias(f"{col_name}_min"),
        max(col(col_name)).alias(f"{col_name}_max")
    ).show()

Resumen para: author_num_games_owned
+----------------------------+---------------------------+-----------------------------+--------------------------+--------------------------+
|author_num_games_owned_count|author_num_games_owned_mean|author_num_games_owned_stddev|author_num_games_owned_min|author_num_games_owned_max|
+----------------------------+---------------------------+-----------------------------+--------------------------+--------------------------+
|                    21747371|         1011300.0967205645|         2.1088291610977833E9|                         0|             4398046511619|
+----------------------------+---------------------------+-----------------------------+--------------------------+--------------------------+

Resumen para: author_num_reviews
+------------------------+-----------------------+-------------------------+----------------------+----------------------+
|author_num_reviews_count|author_num_reviews_mean|author_num_reviews_stddev|author_num_revi

# Particionamiento

Usando las variables de particionamiento
* language
* received_for_free
* recommended

vamos a limitar a los 5 idiomas con mayor frecuencia lo que nos da 20 combinaciones de partición

##Cálculo de las probabilidades de ocurrencia

In [ ]:
# Primer filtrado solo a los 5 idiomas principales
top_languages = ['english', 'schinese', 'russian', 'brazilian', 'spanish']
df_top_lang = df.filter(col("language").isin(top_languages))

# Agrupar por las combinaciones de particionamiento
partition_counts = (
    df_top_lang
    .groupBy("recommended", "received_for_free", "language")
    .count()
    .withColumnRenamed("count", "combinaciones")
)

# Total general para calcular probabilidades
total = df_top_lang.count()

# Añadir columna con probabilidad de ocurrencia
partition_probs = partition_counts.withColumn(
    "probabilidad",
    col("combinaciones") / total
)

partition_probs.orderBy("language", "recommended", "received_for_free").show(20)


+-----------+-----------------+---------+-------------+--------------------+
|recommended|received_for_free| language|combinaciones|        probabilidad|
+-----------+-----------------+---------+-------------+--------------------+
|      false|            false|brazilian|        44881|0.002579345876828174|
|      false|             true|brazilian|         2009|1.154587880516878...|
|       true|            false|brazilian|       756037| 0.04345003272385959|
|       true|             true|brazilian|        34597|0.001988316421216647|
|      false|            false|  english|      1020063|0.058623811705509635|
|      false|             true|  english|        30930|0.001777571087326...|
|       true|            false|  english|      8335990| 0.47907581015977563|
|       true|             true|  english|       248454|0.014278844065004504|
|      false|            false|  russian|       242361| 0.01392867462966407|
|      false|             true|  russian|        11105|6.382129623265274E-4|

In [ ]:
partition_probs

recommended,received_for_free,language,combinaciones,probabilidad
false,true,russian,11105,6.382129623265274E-4
true,false,schinese,2832942,0.1628113737883149
false,true,spanish,1943,1.116657168663163E-4
false,true,english,30930,0.001777571087326...
true,false,spanish,728861,0.0418882069278951
false,false,english,1020063,0.058623811705509635
true,true,brazilian,34597,0.001988316421216647
true,true,schinese,70001,0.004023011758290791
false,false,spanish,56421,0.003242558626512832
true,true,english,248454,0.014278844065004504


# Filtrado de subconjuntos según reglas de particionamiento

## Descripción
En esta sección se filtran subconjuntos del dataset `df_top_lang` conforme a las reglas de particionamiento basadas en:
- Idioma (`language`) — top 5 idiomas
- ¿Recibido gratis? (`received_for_free`) — booleano
- ¿Recomendado? (`recommended`) — booleano

Se generan 20 subconjuntos.

---


In [ ]:
from itertools import product

# Definir los valores posibles de cada dimensión de partición
top_5_languages = ['english', 'schinese', 'russian', 'brazilian', 'spanish']
received_options = [True, False]
recommended_options = [True, False]

# Generar todas las combinaciones posibles
partitions = list(product(top_5_languages, received_options, recommended_options))

# Crear subconjuntos filtrando por cada combinación
df_subsets = {}

for lang, received, recommended in partitions:
    key = f"lang={lang}_free={received}_rec={recommended}"
    filtered_df = df_top_lang.filter(
        (F.col("language") == lang) &
        (F.col("received_for_free") == received) &
        (F.col("recommended") == recommended)
    )
    df_subsets[key] = filtered_df
    count = filtered_df.count()
    print(f"Subset: {key} — Registros: {count}")

# Mostrar ejemplo del contenido de uno de los subconjuntos
sample_key = list(df_subsets.keys())[0]
df_subsets[sample_key].show(5)


Subset: lang=english_free=True_rec=True — Registros: 248454
Subset: lang=english_free=True_rec=False — Registros: 30930
Subset: lang=english_free=False_rec=True — Registros: 8335990
Subset: lang=english_free=False_rec=False — Registros: 1020063
Subset: lang=schinese_free=True_rec=True — Registros: 70001
Subset: lang=schinese_free=True_rec=False — Registros: 17173
Subset: lang=schinese_free=False_rec=True — Registros: 2832942
Subset: lang=schinese_free=False_rec=False — Registros: 844851
Subset: lang=russian_free=True_rec=True — Registros: 96685
Subset: lang=russian_free=True_rec=False — Registros: 11105
Subset: lang=russian_free=False_rec=True — Registros: 1998749
Subset: lang=russian_free=False_rec=False — Registros: 242361
Subset: lang=brazilian_free=True_rec=True — Registros: 34597
Subset: lang=brazilian_free=True_rec=False — Registros: 2009
Subset: lang=brazilian_free=False_rec=True — Registros: 756037
Subset: lang=brazilian_free=False_rec=False — Registros: 44881
Subset: lang=span

In [ ]:
# Prueba de muestreo: 30% sin reemplazo de una partición
sample_30 = df_subsets["lang=english_free=False_rec=True"].sample(withReplacement=False, fraction=0.3)
sample_30.show(5)

+---+------+--------------------+---------+--------+--------------------+-----------------+-----------------+-----------+-------------+-----------+-------------------+-------------+--------------+-----------------+---------------------------+-----------------+----------------------+------------------+-----------------------+------------------------------+-------------------------+------------------+
|_c0|app_id|            app_name|review_id|language|              review|timestamp_created|timestamp_updated|recommended|votes_helpful|votes_funny|weighted_vote_score|comment_count|steam_purchase|received_for_free|written_during_early_access|   author_steamid|author_num_games_owned|author_num_reviews|author_playtime_forever|author_playtime_last_two_weeks|author_playtime_at_review|author_last_played|
+---+------+--------------------+---------+--------+--------------------+-----------------+-----------------+-----------+-------------+-----------+-------------------+-------------+-------------

In [ ]:
# Aplicar muestreo del 30% sin reemplazo a cada partición y validar tamaño
for key, subset_df in df_subsets.items():
    sampled_df = subset_df.sample(withReplacement=False, fraction=0.3, seed=42)
    sampled_count = sampled_df.count()

    print(f"{key} — Muestra (30%): {sampled_count} registros")
    sampled_df.show(5)

lang=english_free=True_rec=True — Muestra (30%): 74761 registros
+----+------+--------------------+---------+--------+--------------------+-----------------+-----------------+-----------+-------------+-----------+-------------------+-------------+--------------+-----------------+---------------------------+-----------------+----------------------+------------------+-----------------------+------------------------------+-------------------------+------------------+
| _c0|app_id|            app_name|review_id|language|              review|timestamp_created|timestamp_updated|recommended|votes_helpful|votes_funny|weighted_vote_score|comment_count|steam_purchase|received_for_free|written_during_early_access|   author.steamid|author.num_games_owned|author.num_reviews|author.playtime_forever|author.playtime_last_two_weeks|author.playtime_at_review|author.last_played|
+----+------+--------------------+---------+--------+--------------------+-----------------+-----------------+-----------+-----

In [ ]:
# =============================================
# Ejemplo 2: Muestreo estratificado con sampleBy()
# =============================================

# Muestreo proporcional por 'language' dentro del subconjunto partition_1
# Primero extraemos un subconjunto base
partition_1 = df.filter(
    (F.col("recommended") == True) &
    (F.col("received_for_free") == False) &
    (F.col("language").isin("english", "russian", "spanish"))
)

# Definir las fracciones deseadas para cada valor de 'language'
fractions = {
    "english": 0.3,
    "russian": 0.3,
    "spanish": 0.3
}

# Aplicar sampleBy sobre 'language'
sampled_partition_1 = partition_1.sampleBy("language", fractions, seed=42)

# Ver muestra
sampled_partition_1.show(5)


+---+------+--------------------+---------+--------+--------------------+-----------------+-----------------+-----------+-------------+-----------+-------------------+-------------+--------------+-----------------+---------------------------+-----------------+----------------------+------------------+-----------------------+------------------------------+-------------------------+------------------+
|_c0|app_id|            app_name|review_id|language|              review|timestamp_created|timestamp_updated|recommended|votes_helpful|votes_funny|weighted_vote_score|comment_count|steam_purchase|received_for_free|written_during_early_access|   author.steamid|author.num_games_owned|author.num_reviews|author.playtime_forever|author.playtime_last_two_weeks|author.playtime_at_review|author.last_played|
+---+------+--------------------+---------+--------+--------------------+-----------------+-----------------+-----------+-------------+-----------+-------------------+-------------+-------------

In [ ]:
# =============================================
# Ejemplo 3: Muestreo aleatorio simple con sample()
# =============================================

# Tomar el 30% de cualquier partición (ejemplo con partition_2)
partition_2 = df.filter(
    (F.col("recommended") == False) &
    (F.col("received_for_free") == True) &
    (F.col("language").isin("english", "russian", "spanish"))
)

# Tomar una muestra aleatoria del 30% sin reemplazo
sampled_partition_2 = partition_2.sample(withReplacement=False, fraction=0.3, seed=42)

# Ver muestra
sampled_partition_2.show(5)


+------+------+--------------------+---------+--------+--------------------+-----------------+-----------------+-----------+-------------+-----------+-------------------+-------------+--------------+-----------------+---------------------------+-----------------+----------------------+------------------+-----------------------+------------------------------+-------------------------+------------------+
|   _c0|app_id|            app_name|review_id|language|              review|timestamp_created|timestamp_updated|recommended|votes_helpful|votes_funny|weighted_vote_score|comment_count|steam_purchase|received_for_free|written_during_early_access|   author.steamid|author.num_games_owned|author.num_reviews|author.playtime_forever|author.playtime_last_two_weeks|author.playtime_at_review|author.last_played|
+------+------+--------------------+---------+--------+--------------------+-----------------+-----------------+-----------+-------------+-----------+-------------------+-------------+----